In [1]:
import os
import re
import json

import numpy as np
import pandas as pd
from tqdm import tqdm

from konlpy.tag import Okt

In [2]:
FILTERS = "([~.,!?\"':;)(])"
PAD = "<PAD>"
STD = "<SOS>"
END = "<END>"
UNK = "<UNK>"

In [3]:
PAD_INDEX = 0
STD_INDEX = 1
END_INDEX = 2
UNK_INDEX = 3

In [4]:
MARKER = [PAD, STD, END, UNK]
CHANGE_FILTER = re.compile(FILTERS)

In [5]:
MAX_SEQUENCE = 25

# Load Data set

In [6]:
def load_data(path):
    data_df = pd.read_csv(path, header=0)
    question, answer = list(data_df['Q']), list(data_df['A'])
    return question, answer

In [7]:
def data_tokenizer(data):
    words = []
    for sentence in data:
        sentence = re.sub(CHANGE_FILTER, "", sentence)
        for word in sentence.split():
            words.append(word)

    return [word for word in words if word]

In [8]:
def prepro_like_morphlized(data):
    morph_analyzer = Okt()
    result_data = list()
    for seq in tqdm(data):
        morphlized_seq = " ".join(morph_analyzer.morphs(seq.replace(' ', '')))
        result_data.append(morphlized_seq)
    return result_data

In [9]:
def make_vocabulary(vocabulary_list):
    word2idx = {word: idx for idx, word in enumerate(vocabulary_list)}
    idx2word = {idx: word for idx, word in enumerate(vocabulary_list)}
    return word2idx, idx2word

In [10]:
def load_vocabulary(path, vocab_path, tokenize_as_morph=False):
    vocabulary_list = []
    if not os.path.exists(vocab_path):
        if os.path.exists(path):
            data_df = pd.read_csv(path, encoding='utf-8')
            question, answer = list(data_df['Q']), list(data_df['A'])
            if tokenize_as_morph:
                question = prepro_like_morphlized(question)
                answer = prepro_like_morphlized(answer)

            data = []
            data.extend(question)
            data.extend(answer)
            words = data_tokenizer(data)
            words = list(set(words))
            words[:0] = MARKER

        with open(vocab_path, 'w', encoding='utf-8') as vocabulary_file:
            for word in words:
                vocabulary_file.write(word + '\n')

    with open(vocab_path, 'r', encoding='utf-8') as vocabulary_file:
        for line in vocabulary_file:
            vocabulary_list.append(line.strip())
    word2idx, idx2word = make_vocabulary(vocabulary_list)

    return word2idx, idx2word, len(word2idx)

In [11]:
PATH = './data_set/ChatBotData.csv'
VOCAB_PATH = './data_set/vocabulary.txt'

In [12]:
inputs, outputs = load_data(PATH)

In [13]:
word2idx, idx2word, vocab_size = load_vocabulary(PATH, VOCAB_PATH, False)

# Encoder Preprocessing layer

In [17]:
def enc_processing(value, dictionary, tokenize_as_morph=False):
    # 인덱스 값들을 가지고 있는
    # 배열이다.(누적된다.)
    sequences_input_index = []

    # 하나의 인코딩 되는 문장의
    # 길이를 가지고 있다.(누적된다.)
    sequences_length = []

    # 형태소 토크나이징 사용 유무
    if tokenize_as_morph:
        value = prepro_like_morphlized(value)

    # 한줄씩 불어온다.
    for sequence in value:
        # FILTERS = "([~.,!?\"':;)(])"
        # 정규화를 사용하여 필터에 들어 있는
        # 값들을 "" 으로 치환 한다.
        sequence = re.sub(CHANGE_FILTER, "", sequence)

        # 하나의 문장을 인코딩 할때
        # 가지고 있기 위한 배열이다.
        sequence_index = []

        # 문장을 스페이스 단위로
        # 자르고 있다.
        for word in sequence.split():
            # 잘려진 단어들이 딕셔너리에 존재 하는지 보고
            # 그 값을 가져와 sequence_index에 추가한다.
            if dictionary.get(word) is not None:
                sequence_index.extend([dictionary[word]])

            # 잘려진 단어가 딕셔너리에 존재 하지 않는
            # 경우 이므로 UNK(2)를 넣어 준다.
            else:
                sequence_index.extend([dictionary[UNK]])
        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 자르고 있다.
        if len(sequence_index) > MAX_SEQUENCE:
            sequence_index = sequence_index[:MAX_SEQUENCE]

        # 하나의 문장에 길이를 넣어주고 있다.
        sequences_length.append(len(sequence_index))

        # max_sequence_length보다 문장 길이가
        # 작다면 빈 부분에 PAD(0)를 넣어준다.
        sequence_index += (MAX_SEQUENCE - len(sequence_index)) * [dictionary[PAD]]

        # 인덱스화 되어 있는 값을
        # sequences_input_index에 넣어 준다.
        sequences_input_index.append(sequence_index)

    # 인덱스화된 일반 배열을 넘파이 배열로 변경한다.
    # 이유는 텐서플로우 dataset에 넣어 주기 위한
    # 사전 작업이다.
    # 넘파이 배열에 인덱스화된 배열과
    # 그 길이를 넘겨준다.
    return np.asarray(sequences_input_index), sequences_length

# Decoder Preprocessing Layer

In [18]:
def dec_output_processing(value, dictionary, tokenize_as_morph=False):
    # 인덱스 값들을 가지고 있는
    # 배열이다.(누적된다)
    sequences_output_index = []

    # 하나의 디코딩 입력 되는 문장의
    # 길이를 가지고 있다.(누적된다)
    sequences_length = []

    # 형태소 토크나이징 사용 유무
    if tokenize_as_morph:
        value = prepro_like_morphlized(value)

    # 한줄씩 불어온다.
    for sequence in value:
        # FILTERS = "([~.,!?\"':;)(])"
        # 정규화를 사용하여 필터에 들어 있는
        # 값들을 "" 으로 치환 한다.
        sequence = re.sub(CHANGE_FILTER, "", sequence)

        # 하나의 문장을 디코딩 할때 가지고
        # 있기 위한 배열이다.
        sequence_index = []

        # 디코딩 입력의 처음에는 START가 와야 하므로
        # 그 값을 넣어 주고 시작한다.
        # 문장에서 스페이스 단위별로 단어를 가져와서 딕셔너리의
        # 값인 인덱스를 넣어 준다.
        sequence_index = [dictionary[STD]] + [dictionary[word] if word in dictionary else dictionary[UNK] for word in sequence.split()]

        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 자르고 있다.
        if len(sequence_index) > MAX_SEQUENCE:
            sequence_index = sequence_index[:MAX_SEQUENCE]

        # 하나의 문장에 길이를 넣어주고 있다.
        sequences_length.append(len(sequence_index))

        # max_sequence_length보다 문장 길이가
        # 작다면 빈 부분에 PAD(0)를 넣어준다.
        sequence_index += (MAX_SEQUENCE - len(sequence_index)) * [dictionary[PAD]]

        # 인덱스화 되어 있는 값을
        # sequences_output_index 넣어 준다.
        sequences_output_index.append(sequence_index)

    # 인덱스화된 일반 배열을 넘파이 배열로 변경한다.
    # 이유는 텐서플로우 dataset에 넣어 주기 위한
    # 사전 작업이다.
    # 넘파이 배열에 인덱스화된 배열과 그 길이를 넘겨준다.
    return np.asarray(sequences_output_index), sequences_length

In [19]:
def dec_target_processing(value, dictionary, tokenize_as_morph=False):
    # 인덱스 값들을 가지고 있는
    # 배열이다.(누적된다)
    sequences_target_index = []

     # 형태소 토크나이징 사용 유무
    if tokenize_as_morph:
        value = prepro_like_morphlized(value)

    # 한줄씩 불어온다.
    for sequence in value:
        # FILTERS = "([~.,!?\"':;)(])"
        # 정규화를 사용하여 필터에 들어 있는
        # 값들을 "" 으로 치환 한다.
        sequence = re.sub(CHANGE_FILTER, "", sequence)

        # 문장에서 스페이스 단위별로 단어를 가져와서
        # 딕셔너리의 값인 인덱스를 넣어 준다.
        # 디코딩 출력의 마지막에 END를 넣어 준다.
        sequence_index = [dictionary[word] if word in dictionary else dictionary[UNK] for word in sequence.split()]

        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 자르고 있다.
        # 그리고 END 토큰을 넣어 준다
        if len(sequence_index) >= MAX_SEQUENCE:
            sequence_index = sequence_index[:MAX_SEQUENCE - 1] + [dictionary[END]]
        else:
            sequence_index += [dictionary[END]]

        # max_sequence_length보다 문장 길이가
        # 작다면 빈 부분에 PAD(0)를 넣어준다.
        sequence_index += (MAX_SEQUENCE - len(sequence_index)) * [dictionary[PAD]]

        # 인덱스화 되어 있는 값을
        # sequences_target_index에 넣어 준다.
        sequences_target_index.append(sequence_index)

    # 인덱스화된 일반 배열을 넘파이 배열로 변경한다.
    # 이유는 텐서플로우 dataset에 넣어 주기 위한 사전 작업이다.
    # 넘파이 배열에 인덱스화된 배열과 그 길이를 넘겨준다.
    return np.asarray(sequences_target_index)

# DATA PREPROCESSING

In [29]:
inputs, outputs = load_data(PATH)

In [30]:
char2idx, idx2char, vocab_size = load_vocabulary(PATH, VOCAB_PATH, tokenize_as_morph=False)

In [31]:
vocab_size

20705

In [32]:
char2idx

{'<PAD>': 0,
 '<SOS>': 1,
 '<END>': 2,
 '<UNK>': 3,
 '밤샜어': 4,
 '생각나네': 5,
 '잘자': 6,
 '밤이네요': 7,
 '세달': 8,
 '사로잡는': 9,
 '새벽': 10,
 '챙김받고': 11,
 '좋겠는': 12,
 '쇼핑했더니': 13,
 '붙잡아도': 14,
 '만들어줄까': 15,
 '울렸으면': 16,
 '냉정해진다': 17,
 '휘성노래가': 18,
 '톡을': 19,
 '아니지요': 20,
 '실행해보세요': 21,
 '걱정임': 22,
 '차였어': 23,
 '봐달라고': 24,
 '나른하다': 25,
 '이야기해주세요': 26,
 '휴반년이라는': 27,
 '3개월만에': 28,
 '듣고': 29,
 '결혼하재': 30,
 '쓴만큼': 31,
 '오려나': 32,
 '마지막으로': 33,
 '위험하네요': 34,
 '이상형뿐일': 35,
 '썼다가': 36,
 '기적같은': 37,
 '따르나봐요': 38,
 '양해를': 39,
 '않은거예요': 40,
 '허전하신가봐요': 41,
 '배웠길': 42,
 '한살': 43,
 '방금다녀와서': 44,
 '이사를': 45,
 '사람에게': 46,
 '첫만남': 47,
 '확인사살': 48,
 '정리해야할': 49,
 '책상': 50,
 '어릴땐': 51,
 '자격증': 52,
 '잔인했던가': 53,
 '미팅에서': 54,
 '울어버려요': 55,
 '통장': 56,
 '면접': 57,
 '욕은': 58,
 '잊혀질거라': 59,
 '바라는것도': 60,
 '빠졌나봐요': 61,
 '생각들이': 62,
 '없답니다': 63,
 '투자한': 64,
 '기억안': 65,
 '행복': 66,
 '야식': 67,
 '택배': 68,
 '답답하겠네요': 69,
 '거랑': 70,
 '답답한게': 71,
 '맞춰보는': 72,
 '귀찮게해': 73,
 '배울': 74,
 '생각': 75,
 '모른': 76,
 '멀리하세요': 77,
 '연속이예요': 

In [33]:
idx2char

{0: '<PAD>',
 1: '<SOS>',
 2: '<END>',
 3: '<UNK>',
 4: '밤샜어',
 5: '생각나네',
 6: '잘자',
 7: '밤이네요',
 8: '세달',
 9: '사로잡는',
 10: '새벽',
 11: '챙김받고',
 12: '좋겠는',
 13: '쇼핑했더니',
 14: '붙잡아도',
 15: '만들어줄까',
 16: '울렸으면',
 17: '냉정해진다',
 18: '휘성노래가',
 19: '톡을',
 20: '아니지요',
 21: '실행해보세요',
 22: '걱정임',
 23: '차였어',
 24: '봐달라고',
 25: '나른하다',
 26: '이야기해주세요',
 27: '휴반년이라는',
 28: '3개월만에',
 29: '듣고',
 30: '결혼하재',
 31: '쓴만큼',
 32: '오려나',
 33: '마지막으로',
 34: '위험하네요',
 35: '이상형뿐일',
 36: '썼다가',
 37: '기적같은',
 38: '따르나봐요',
 39: '양해를',
 40: '않은거예요',
 41: '허전하신가봐요',
 42: '배웠길',
 43: '한살',
 44: '방금다녀와서',
 45: '이사를',
 46: '사람에게',
 47: '첫만남',
 48: '확인사살',
 49: '정리해야할',
 50: '책상',
 51: '어릴땐',
 52: '자격증',
 53: '잔인했던가',
 54: '미팅에서',
 55: '울어버려요',
 56: '통장',
 57: '면접',
 58: '욕은',
 59: '잊혀질거라',
 60: '바라는것도',
 61: '빠졌나봐요',
 62: '생각들이',
 63: '없답니다',
 64: '투자한',
 65: '기억안',
 66: '행복',
 67: '야식',
 68: '택배',
 69: '답답하겠네요',
 70: '거랑',
 71: '답답한게',
 72: '맞춰보는',
 73: '귀찮게해',
 74: '배울',
 75: '생각',
 76: '모른',
 77: '멀리하세요',
 78: '연속이예

In [34]:
index_inputs, input_seq_len = enc_processing(inputs, char2idx, tokenize_as_morph=False)
index_outputs, output_seq_len = dec_output_processing(outputs, char2idx, tokenize_as_morph=False)
index_targets = dec_target_processing(outputs, char2idx, tokenize_as_morph=False)

In [35]:
index_inputs

array([[11034,  4489,     0, ...,     0,     0,     0],
       [ 3278, 11766, 16612, ...,     0,     0,     0],
       [ 4584, 15569, 13314, ...,     0,     0,     0],
       ...,
       [ 1245,  4083, 19327, ...,     0,     0,     0],
       [17844, 11697,  8357, ...,     0,     0,     0],
       [ 9695, 12475,     0, ...,     0,     0,     0]])

In [36]:
input_seq_len

[2,
 3,
 3,
 4,
 2,
 2,
 2,
 4,
 7,
 4,
 5,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 3,
 4,
 4,
 3,
 4,
 2,
 3,
 2,
 3,
 2,
 2,
 3,
 3,
 2,
 2,
 3,
 3,
 1,
 4,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 4,
 2,
 4,
 3,
 2,
 4,
 3,
 4,
 2,
 2,
 2,
 3,
 3,
 2,
 2,
 5,
 5,
 3,
 3,
 6,
 4,
 3,
 5,
 4,
 2,
 3,
 4,
 4,
 6,
 4,
 5,
 4,
 8,
 2,
 4,
 1,
 1,
 2,
 2,
 2,
 3,
 2,
 1,
 4,
 4,
 4,
 1,
 1,
 2,
 3,
 4,
 5,
 5,
 1,
 2,
 5,
 3,
 3,
 3,
 5,
 2,
 3,
 2,
 2,
 2,
 4,
 4,
 2,
 5,
 2,
 2,
 1,
 2,
 1,
 3,
 5,
 1,
 3,
 4,
 4,
 5,
 2,
 2,
 3,
 2,
 3,
 3,
 2,
 4,
 3,
 2,
 3,
 4,
 2,
 5,
 3,
 4,
 3,
 2,
 2,
 3,
 3,
 3,
 5,
 3,
 7,
 4,
 4,
 4,
 4,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 3,
 3,
 4,
 3,
 3,
 4,
 5,
 3,
 3,
 3,
 3,
 2,
 3,
 4,
 4,
 3,
 2,
 3,
 2,
 3,
 4,
 2,
 3,
 3,
 3,
 2,
 3,
 3,
 4,
 3,
 3,
 6,
 3,
 1,
 3,
 3,
 2,
 2,
 3,
 2,
 2,
 3,
 3,
 4,
 5,
 3,
 2,
 2,
 3,
 2,
 2,
 3,
 4,
 3,
 3,
 4,
 3,
 3,
 5,
 3,
 6,
 5,
 2,
 2,
 3,
 3,
 2,
 3,
 3,
 3,
 2,
 3,
 1,
 3,
 4,
 4,
 2,
 2,
 3,
 2,
 5,
 3,
 3,
 2,


In [37]:
data_configs = {}
data_configs['char2idx'] = char2idx
data_configs['idx2char'] = idx2char
data_configs['vocab_size'] = vocab_size
data_configs['pad_symbol'] = PAD
data_configs['std_symbol'] = STD
data_configs['end_symbol'] = END
data_configs['unk_symbol'] = UNK

In [40]:
DATA_IN_PATH = './data_set/'
TRAIN_INPUTS = 'train_inputs.npy'
TRAIN_OUTPUTS = 'train_outputs.npy'
TRAIN_TARGETS = 'train_targets.npy'
DATA_CONFIGS = 'data_configs.json'

In [41]:
np.save(open(DATA_IN_PATH + TRAIN_INPUTS, 'wb'), index_inputs)
np.save(open(DATA_IN_PATH + TRAIN_OUTPUTS , 'wb'), index_outputs)
np.save(open(DATA_IN_PATH + TRAIN_TARGETS , 'wb'), index_targets)

json.dump(data_configs, open(DATA_IN_PATH + DATA_CONFIGS, 'w'))